In [1]:
pip install comtypes


Note: you may need to restart the kernel to use updated packages.


In [ ]:
"""
mic_hotkeys_cominit_fixed.py - MODIFIED FOR SYSTEM VOLUME CONTROL
Global hotkeys to control SYSTEM (Multimedia playback) volume (Windows).
This version ensures COM is initialized inside each hotkey handler thread.

Hotkeys:
  Ctrl+Alt+Up    -> increase system volume by 5%
  Ctrl+Alt+Down  -> decrease system volume by 5%
  Ctrl+Alt+M     -> toggle mute/unmute
  Ctrl+Alt+Q     -> stop hotkeys (clean shutdown)
"""
import sys
import time
import platform
import threading
from ctypes import POINTER, cast
from comtypes import CLSCTX_ALL, CoInitialize, CoUninitialize
from comtypes.client import CreateObject
from comtypes import GUID
from functools import wraps

# --- Platform Check ---
if platform.system() != "Windows":
    raise SystemExit("This script runs only on Windows.")

# --- 3rd-party libraries (Install using: pip install keyboard pycaw comtypes) ---
try:
    import keyboard
except Exception:
    raise SystemExit("Install required package: pip install keyboard")

try:
    from pycaw.pycaw import IAudioEndpointVolume, IMMDeviceEnumerator
except Exception:
    raise SystemExit("Install required packages: pip install pycaw comtypes")

# --- Constants ---
eRender = 0
# *** MODIFIED: Changed the device role to eMultimedia (1) for system volume control ***
eMultimedia = 1 
STEP_PERCENT = 5.0

# --- Stop event for clean shutdown ---
stop_event = threading.Event()

# --- Helper: create IMMDeviceEnumerator (typed) ---
def _create_mmdevice_enumerator():
    """Returns a COM object for enumerating audio devices."""
    try:
        # Preferred method using ProgID
        return CreateObject("MMDeviceEnumerator.MMDeviceEnumerator", interface=IMMDeviceEnumerator)
    except Exception:
        # Fallback using GUID for the MMDeviceEnumerator class
        clsid = GUID("{BCDE0395-E52F-467C-8E3D-C4579291692E}")
        return CreateObject(clsid, interface=IMMDeviceEnumerator)

def _get_volume_interface_for_default():
    """
    Returns an IAudioEndpointVolume pointer for the default RENDER (System/Multimedia) device.
    Caller must ensure COM is initialized in the calling thread.
    """
    enumerator = _create_mmdevice_enumerator()
    # Get the default device for rendering (eRender) using the Multimedia role (eMultimedia)
    default_device = enumerator.GetDefaultAudioEndpoint(eRender, eMultimedia) 
    # Activate the volume interface on that device
    iface = default_device.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    # Return a correctly typed pointer
    return cast(iface, POINTER(IAudioEndpointVolume))

# --- Scalar/Percentage Conversion Helpers ---
def _percent_to_scalar(p):
    """Converts a percentage (0-100) to a scalar (0.0-1.0)."""
    return max(0.0, min(1.0, p / 100.0))

def _scalar_to_percent(s):
    """Converts a scalar (0.0-1.0) to a percentage (0-100)."""
    return max(0.0, min(100.0, s * 100.0))

# --- Decorator to ensure COM initialized per-thread for hotkey handlers ---
def ensure_com(func):
    """
    Decorator that initializes and uninitializes COM for the duration of the
    decorated function's execution.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        CoInitialize()
        try:
            return func(*args, **kwargs)
        except Exception:
            # Print traceback for easier debugging
            import traceback
            print("Exception in handler:", file=sys.stderr)
            traceback.print_exc()
        finally:
            # Uninitialize COM in this thread when handler completes
            try:
                CoUninitialize()
            except Exception:
                pass
    return wrapper

# --- Hotkey Handlers ---
@ensure_com
def increase_volume():
    """Increases system volume by STEP_PERCENT."""
    vol = _get_volume_interface_for_default()
    # CORRECTED LINE: Added Scalar()
    cur = float(vol.GetMasterVolumeLevelScalar()) 
    cur_pct = _scalar_to_percent(cur)
    new_pct = min(100.0, cur_pct + STEP_PERCENT)
    vol.SetMasterVolumeLevelScalar(_percent_to_scalar(new_pct), None)
    print(f"[+] System volume -> {new_pct:.0f}%")

@ensure_com
def decrease_volume():
    """Decreases system volume by STEP_PERCENT."""
    vol = _get_volume_interface_for_default()
    # CORRECTED LINE: Added Scalar()
    cur = float(vol.GetMasterVolumeLevelScalar())
    cur_pct = _scalar_to_percent(cur)
    new_pct = max(0.0, cur_pct - STEP_PERCENT)
    vol.SetMasterVolumeLevelScalar(_percent_to_scalar(new_pct), None)
    print(f"[-] System volume -> {new_pct:.0f}%")

@ensure_com
def toggle_mute():
    """Toggles the mute state of the system volume."""
    vol = _get_volume_interface_for_default()
    cur_mute = bool(vol.GetMute())
    new_mute = not cur_mute
    vol.SetMute(1 if new_mute else 0, None)
    # Print the new state (M when muted, U when unmuted)
    print(f"[{'M' if new_mute else 'U'}] System muted -> {new_mute}")

def quit_program():
    """Signal the main loop to stop and unhook hotkeys."""
    print("Stopping hotkeys (quit requested)...")
    try:
        keyboard.unhook_all_hotkeys()
    except Exception:
        pass
    stop_event.set()

# --- Hotkey Registration ---
def register_hotkeys():
    """Unregisters any existing hotkeys and registers the new set."""
    # Ensure a clean slate
    try:
        keyboard.unhook_all_hotkeys()
    except Exception:
        pass

    keyboard.add_hotkey('ctrl+alt+up', increase_volume)
    keyboard.add_hotkey('ctrl+alt+down', decrease_volume)
    keyboard.add_hotkey('ctrl+alt+m', toggle_mute)
    keyboard.add_hotkey('ctrl+alt+q', quit_program)

# --- Main Execution Block ---
if __name__ == "__main__":
    print("Registering hotkeys...")
    register_hotkeys()

    # Try to print initial state (initialize COM briefly here)
    CoInitialize()
    try:
        try:
            vol = _get_volume_interface_for_default()
            # CORRECTED LINE: Added Scalar()
            print(
                f"Initial system volume: {_scalar_to_percent(float(vol.GetMasterVolumeLevelScalar())):.0f}%"
                f"  muted={bool(vol.GetMute())}"
            )
        except Exception as e:
            print("Could not read initial system state:", e)
    finally:
        CoUninitialize()

    print("Hotkeys active. Press Ctrl+Alt+Up/Down to change system volume, Ctrl+Alt+M to toggle mute, Ctrl+Alt+Q to quit.")
    try:
        # Main loop: waits until quit_program() sets stop_event
        while not stop_event.wait(timeout=0.1):
            pass
    except KeyboardInterrupt:
        print("Interrupted by user; exiting.")
    finally:
        # Clean up hotkeys on exit
        try:
            keyboard.unhook_all_hotkeys()
        except Exception:
            pass
        print("Exited system volume hotkeys.")
